Environment: napari_sparrow

In [1]:
%load_ext autoreload
%autoreload 2

### Import packages

In [ ]:
import sparrow as sp
import os
import spatialdata as sd
import cv2 as cv2
import spatialdata as sd
import pandas as pd

### Set paths and create folders

In [3]:
# Specify path to root folder
root_folder = r'D:\Data\2023-07-CarolineAsselman-FIm\Analysis_v2'

# Specify ROI name
ROI = 'A1_ROI1'

# Path to input folders
sdata_path = os.path.join(root_folder, 'output', ROI, 'downstream_analysis')

# Path to output folders
output_path = os.path.join(root_folder, 'analysis_seurat', ROI)
plots_path = os.path.join(output_path, 'plots_3')
os.makedirs(output_path, exist_ok = True)
os.makedirs(plots_path, exist_ok = True)

### Create or read sdata

In [ ]:
sdata = sd.read_zarr(os.path.join(sdata_path, "sdata.zarr"))
sdata

### Read in seurat clusters

In [ ]:
df_before = pd.read_excel(os.path.join(output_path, "ImmuneObj_cells.xlsx"), sheet_name='BeforeFiltering')
df_before.rename(columns={'cells': 'cell_id', 'cluster': 'clusters_before'}, inplace=True)
df_before = df_before[['cell_id', 'clusters_before']]
df_before['clusters_before'] = df_before['clusters_before'].astype(str)
df_before = df_before.set_index('cell_id')
df_before.index = df_before.index.astype(str)
df_before

In [ ]:
df_after = pd.read_excel(os.path.join(output_path, "ImmuneObj_cells.xlsx"), sheet_name='AfterFiltering')
df_after.rename(columns={'cells': 'cell_id', 'cluster': 'clusters_after'}, inplace=True)
df_after = df_after[['cell_id', 'clusters_after']]
df_after['clusters_after'] = df_after['clusters_after'].astype(str)
df_after = df_after.set_index('cell_id')
df_after.index = df_after.index.astype(str)
df_after

### Add seurat clusters to anndata table

In [18]:
sdata.table.obs = sdata.table.obs.merge(df_before, left_index=True, right_index=True, how='outer')
sdata.table.obs = sdata.table.obs.merge(df_after, left_index=True, right_index=True, how='outer')

### Plots for immune cells

In [20]:
from matplotlib.colors import ListedColormap

# Define the colors in a dictionary format
colors_dict_before = {
    '0': "#FBF955",
    '1': "#1AFF1A",
    '2': "#7AE4FB",
    '3': "#FB64F1",
}

colors_dict_after = {
    '0': "#1AFF1A",
    '1': "#7AE4FB",
    '2': "#FB64F1",
}

# Create a custom colormap
cmap_sub_before = ListedColormap([colors_dict_before[key] for key in sorted(colors_dict_before.keys())])
cmap_sub_after = ListedColormap([colors_dict_after[key] for key in sorted(colors_dict_after.keys())])


In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'cluster_before.png'),
    alpha=1,
    cmap=cmap_sub_before,
    column='clusters_before',
    figsize=(17.5, 11.25)
)

sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'cluster_after.png'),
    alpha=1,
    cmap=cmap_sub_after,
    column='clusters_after',
    figsize=(17.5, 11.25)
)